## Feature Engineering

Now that have covered the EDA phase, lets move to some simple feature engineering tasks prior to modeling.

A few things to note:

* It is good practice to split the data prior to augmenting it
* Ill drop the RID column for now

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline
from feature_engine.imputation import MeanMedianImputer

# Import dataset
X_train = pd.read_csv("C:/Users/steve/Desktop/Notebooks/Thesis-Project/Datasets/processed/X_train_processed.csv")
X_test = pd.read_csv("C:/Users/steve/Desktop/Notebooks/Thesis-Project/Datasets/processed/X_test_processed.csv")
y_train = pd.read_csv("C:/Users/steve/Desktop/Notebooks/Thesis-Project/Datasets/processed/y_train_processed.csv")
y_test = pd.read_csv("C:/Users/steve/Desktop/Notebooks/Thesis-Project/Datasets/processed/y_test_processed.csv")

In [ ]:
sc = StandardScaler()

# let's transform the data 
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape

((425, 22), (183, 22))

### Pipeline: Data imputation & feature scaling

Lets extract all columns with missing values

In [27]:
na_columns = dataset.columns[dataset.isnull().sum() > 0]
na_columns

Index(['MMSE0m', 'HipsASMbaseline', 'HipsContrastbaseline',
       'HipsCorelationbaseline', 'HipsVariancebaseline',
       'HipsSumAveragebaseline', 'HipsSumVariancebaseline',
       'HipsEntropybaseline', 'HipsClusterShadebaseline', 'ERCsASMbaseline',
       'ERCsContrastbaseline', 'ERCsCorelationbaseline',
       'ERCsVariancebaseline', 'ERCsSumAveragebaseline',
       'ERCsSumVariancebaseline', 'ERCsEntropybaseline',
       'ERCsClusterShadebaseline', 'ERCs_thicknessbaseline',
       'ERCsVolumebaseline', 'HipposcampusVolumebaseline'],
      dtype='object')

In [28]:
pipe = Pipeline([
    ("imputer", MeanMedianImputer(
        imputation_method="mean", 
        variables=[
            'MMSE0m', 'HipsASMbaseline', 'HipsContrastbaseline',
            'HipsCorelationbaseline', 'HipsVariancebaseline',
            'HipsSumAveragebaseline', 'HipsSumVariancebaseline',
            'HipsEntropybaseline', 'HipsClusterShadebaseline', 
            'ERCsASMbaseline', 'ERCsContrastbaseline', 
            'ERCsCorelationbaseline', 'ERCsVariancebaseline', 
            'ERCsSumAveragebaseline', 'ERCsSumVariancebaseline',
            'ERCsEntropybaseline', 'ERCsClusterShadebaseline', 
            'ERCs_thicknessbaseline', 'ERCsVolumebaseline', 
            'HipposcampusVolumebaseline'
        ]
    )),
    ("scaler", StandardScaler().set_output(transform="pandas")),
])

pipe.fit(X_train)

# let's transform the data with the pipeline
X_train_scaled = pipe.transform(X_train)
X_test_scaled = pipe.transform(X_test)

**Note**

* Typically, after imputing the dataset, we analyze and visualize the data to assess whether it has been significantly affected. However, for the sake of simplicity, we will temporarily skip this step.
* A split analysis will be conducted after the first iteration.
* Additionally, no feature selection will be performed until we complete the first iteration.
* Variables have not been dropped.
* Hyperparameter optimization has not been performed.

## Model Selection

We will experiment with various models that were previously mentioned in paper. 

* Logistic regression
* Support vector machine
* Decision tree
* Random forest

I will only focus on these 4 models for now. Though i would love to check how a simple ANN would work here. I'll try that afterwards.

Documentation on sklearn for any of the below models
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

lg = LogisticRegression(multi_class = "multinomial", solver = "lbfgs", max_iter = 1000, random_state = 42)

svm = SVC(kernel ='rbf', decision_function_shape ='ovo', probability = True, random_state = 42)

dt = DecisionTreeClassifier(criterion ='gini', max_depth = 5, min_samples_split = 10, 
                            min_samples_leaf = 5, max_features = 'sqrt', random_state = 42)

rf = RandomForestClassifier(n_estimators = 100, criterion = 'gini', max_depth = 5, min_samples_split = 10, 
                            min_samples_leaf = 5, max_features = 'sqrt', bootstrap = True, random_state = 42)

In the paper, it was stated that they performed 5 KFolds, so we will replicate their approach. 

ROC AUC along with other mentioned metrics will be covered here

In [34]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import KFold, cross_validate

kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

# Define metrics to evaluate
scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average = 'weighted', zero_division=0),
    'recall': make_scorer(recall_score, average = 'weighted', zero_division=0),
    'f1': make_scorer(f1_score, average = 'weighted', zero_division=0),
    'roc_auc': make_scorer(roc_auc_score, multi_class='ovr', response_method = "predict_proba")
}

models  = {"Logistic Regression": lg, 
           "Support Vector Machine": svm, 
           "Decision Tree": dt, 
           "Random Forest": rf
}

model_data_mapping = {
    'Logistic Regression': X_train_scaled,
    'Support Vector Machine': X_train_scaled,
    'Decision Tree': X_train,
    'Random Forest': X_train
}

Now lets check the metrics

In [35]:
for model_name, model in models.items():

    X_train_to_use = model_data_mapping[model_name]
    
    results = cross_validate(model, 
                             X_train_to_use, 
                             y_train, 
                             scoring = scoring_metrics,
                             return_train_score = True,
                             cv = kf)
    print("------------------------------------------------------")
    print(model_name)
    for metric in scoring_metrics.keys():
            print(f'Mean train {metric}:', np.mean(results[f'train_{metric}']), '±', np.std(results[f'train_{metric}']))
            print(f'Mean test {metric}:', np.mean(results[f'test_{metric}']), '±', np.std(results[f'test_{metric}']))

------------------------------------------------------
Logistic Regression
Mean train accuracy: 0.7176470588235294 ± 0.010685824779167581
Mean test accuracy: 0.628235294117647 ± 0.021820278812931068
Mean train precision: 0.7137715549459933 ± 0.009994613394817947
Mean test precision: 0.6346294513584386 ± 0.016375364642513202
Mean train recall: 0.7176470588235294 ± 0.010685824779167581
Mean test recall: 0.628235294117647 ± 0.021820278812931068
Mean train f1: 0.7141321948311611 ± 0.01008676011818872
Mean test f1: 0.6264786751157274 ± 0.018573953162226077
Mean train roc_auc: 0.9092337235320909 ± 0.005358305500420103
Mean test roc_auc: 0.8598573511351006 ± 0.021712265716621503
------------------------------------------------------
Support Vector Machine
Mean train accuracy: 0.8023529411764706 ± 0.005060191333554468
Mean test accuracy: 0.5999999999999999 ± 0.034899757586332535
Mean train precision: 0.8076762897810934 ± 0.004279107619041377
Mean test precision: 0.5951572225624752 ± 0.06058994

It appears that based on the accuracy metric only, Random Forest can be considered as the best model with a mean test accuracy of 0.6329.
While the worst Model would be Decision Tree, with a mean test accuracy of 0.4847.